In [1]:
import os
import numpy as np
import cv2
import imageio.v2 as imageio
import matplotlib.pyplot as plt

In [2]:
# list all the files in the dataset directory
mask_path = "/home/asif/DBMS-II/SkinDetection/Mask"
non_mask_path = "/home/asif/DBMS-II/SkinDetection/NonMask"
mask_list = os.listdir(mask_path) # skin
non_mask_list = os.listdir(non_mask_path) # non skin

In [3]:
# to randomize and shuffel the lists indices
import random
random.seed(5)

In [8]:
total_images = 555
number_of_train_images = 500
number_of_test_images = 55
indices = ["%04d" % x for x in range(total_images)]

In [5]:
TP = []
TN = []
FP = []
FN = []

In [6]:
# we will do crossfold validation
# we will 10 fold and will do 90% for training and 10% for testing

for i in range(1):
    random.shuffle(indices)
    track_image = np.zeros(shape=(total_images))

    # first train the training set
    n = 256
    skinPixel = np.zeros(shape=(n, n, n))
    nonSkinPixel = np.zeros(shape=(n, n, n))
    skinPixelSum = 0
    nonSkinPixelSum = 0
    skinProbability = np.zeros(shape=(n, n, n))
    nonSkinProbability = np.zeros(shape=(n, n, n))
    learning_ratio = np.zeros(shape=(n, n, n)) 

    for index in range(number_of_train_images):
        track_image[int(indices[index])] = 1 
        mask_image = cv2.imread("/home/asif/DBMS-II/SkinDetection/Mask/"+str(indices[index])+".bmp")
        unmask_image = cv2.imread("/home/asif/DBMS-II/SkinDetection/NonMask/"+str(indices[index])+".jpg")

        height, width, channel = mask_image.shape

        for x in range(height):
            for y in range(width):
                blue = mask_image[x, y, 0]
                green = mask_image[x, y, 1]
                red = mask_image[x, y, 2]

                if (blue > 250 and green > 250 and red > 250):  
                    blue = unmask_image[x, y, 0]
                    green = unmask_image[x, y, 1]
                    red = unmask_image[x, y, 2]

                    nonSkinPixel[red, green, blue] += 1
                    nonSkinPixelSum += 1
                else:
                    skinPixel[red, green, blue] += 1
                    skinPixelSum += 1

        print("Image ", index, " - DONE!")
    
    # now calculate the probabilities
    for r in range(n):
        for g in range(n):
            for b in range(n):
                skinProbability[r, g, b] = skinPixel[r, g, b]/skinPixelSum
                nonSkinProbability[r, g, b] = nonSkinPixel[r, g, b]/nonSkinPixelSum

                if (nonSkinPixel[r, g, b] != 0):
                    learning_ratio[r, g, b] = skinProbability[r, g, b] / nonSkinProbability[r, g, b]
                else:
                    learning_ratio[r, g, b] = 0.0

    print("Training "+str(i)+" done")

    # now test
    T = 0.4
    true_skin = 0
    true_non_skin = 0
    false_skin = 0
    false_non_skin = 0

    for index in indices:
        if track_image[int(index)] != 1:
            test_non_mask_image_path = "/home/asif/DBMS-II/SkinDetection/NonMask/"+str(index)+".jpg"
            #test_image = cv2.imread(test_image_path)
            test_non_mask_image = imageio.imread(test_non_mask_image_path)
            height, width, channel = test_non_mask_image.shape 

            test_mask_image_path = "/home/asif/DBMS-II/SkinDetection/Mask/"+str(index)+".bmp"
            test_mask_image = imageio.imread(test_mask_image_path)

            for x in range(height):
                for y in range(width):
                    r=test_non_mask_image[x][y][0]
                    g=test_non_mask_image[x][y][1]
                    b=test_non_mask_image[x][y][2]
                    
                    red =test_mask_image[x][y][0]
                    green =test_mask_image[x][y][1]
                    blue =test_mask_image[x][y][2]

                    if abs(learning_ratio[r][g][b])<T:  # non skin
                        if red>250 and green>250 and blue>250:
                            true_non_skin = true_non_skin + 1
                        else:
                            false_non_skin = false_non_skin + 1
                    else: # skin
                        if  red<250 and green<250 and blue<250:
                            true_skin = true_skin + 1
                        else:
                            false_skin = false_skin + 1
        

    #print
    print(true_skin, false_skin, true_non_skin, false_non_skin)
    # now store the results in file
    TP.append(true_skin)
    TN.append(true_non_skin)
    FP.append(false_skin)
    FN.append(false_non_skin)

    print("Testing "+str(i)+" done")

# Now store the results in file
with open('TP1.txt', 'w') as filehandle:
    for listitem in TP:
        filehandle.write(f'{listitem}\n')

with open('TN1.txt', 'w') as filehandle:
    for listitem in TN:
        filehandle.write(f'{listitem}\n')

with open('FP1.txt', 'w') as filehandle:
    for listitem in FP:
        filehandle.write(f'{listitem}\n')

with open('FN1.txt', 'w') as filehandle:
    for listitem in FN:
        filehandle.write(f'{listitem}\n')


Image  0  - DONE!
Image  1  - DONE!
Image  2  - DONE!
Image  3  - DONE!
Image  4  - DONE!
Image  5  - DONE!
Image  6  - DONE!
Image  7  - DONE!
Image  8  - DONE!
Image  9  - DONE!
Image  10  - DONE!
Image  11  - DONE!
Image  12  - DONE!
Image  13  - DONE!
Image  14  - DONE!
Image  15  - DONE!
Image  16  - DONE!
Image  17  - DONE!
Image  18  - DONE!
Image  19  - DONE!
Image  20  - DONE!
Image  21  - DONE!
Image  22  - DONE!
Image  23  - DONE!


KeyboardInterrupt: 